# Amazon SageMaker Feature Store Demo
## Complete Guide: Data Transformation, Feature Store Creation, and Data Ingestion

This comprehensive demo shows how to:
1. Transform raw customer and order data for ML use cases
2. Create feature groups in Amazon SageMaker Feature Store
3. Add metadata and descriptions to features
4. Ingest data into the feature store (both online and offline stores)
5. Validate and retrieve data from the feature store

Prerequisites:
- AWS account with SageMaker permissions
- SageMaker notebook instance or SageMaker Studio
- Raw CSV files: customers.csv and orders.csv

## Step 1: Transform Customers Data

In [151]:
import pandas as pd
from sklearn.preprocessing import  LabelEncoder,MinMaxScaler
from datetime import datetime,date,timezone

In [152]:
customers_df = pd.read_csv("raw/customers.csv")

In [153]:
customers_df.head()

,Unnamed: 0,customer_id,name,sex,state,age,is_married,active_since,event_time
0,0,C1,brooke williams,F,alabama,76,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z
1,1,C2,jim reese,M,oregon,76,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z
2,2,C3,adam walker phd,M,north dakota,40,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z
3,3,C4,nathan roberts,M,louisiana,68,False,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z
4,4,C5,richard adkins,M,idaho,62,False,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z


In [154]:
label_encoder = LabelEncoder()
min_max_scaler = MinMaxScaler()

In [155]:
customers_df.drop(columns = ['name','state'], inplace=True)

In [156]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    10000 non-null  int64 
 1   customer_id   10000 non-null  object
 2   sex           10000 non-null  object
 3   age           10000 non-null  int64 
 4   is_married    10000 non-null  bool  
 5   active_since  10000 non-null  object
 6   event_time    10000 non-null  object
dtypes: bool(1), int64(2), object(4)
memory usage: 478.6+ KB


In [157]:
bins = [18, 30, 40, 50, 60, 70, 90]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']
customers_df['age_range'] = pd.cut(customers_df['age'], bins, labels=labels, include_lowest=True)

In [158]:
customers_df.head()

,Unnamed: 0,customer_id,sex,age,is_married,active_since,event_time,age_range
0,0,C1,F,76,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,70-plus
1,1,C2,M,76,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,70-plus
2,2,C3,M,40,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,30-39
3,3,C4,M,68,False,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,60-69
4,4,C5,M,62,False,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,60-69


In [159]:
df_age_group = pd.get_dummies(customers_df['age_range'],prefix="age", dtype='int')

In [160]:
df_age_group.head()

,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,1,0,0,0,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0


In [161]:
customers_df = pd.concat([customers_df,df_age_group],axis=1)
customers_df.head()

,Unnamed: 0,customer_id,sex,age,is_married,active_since,event_time,age_range,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,0,C1,F,76,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,70-plus,0,0,0,0,0,1
1,1,C2,M,76,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,70-plus,0,0,0,0,0,1
2,2,C3,M,40,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,30-39,0,1,0,0,0,0
3,3,C4,M,68,False,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,60-69,0,0,0,0,1,0
4,4,C5,M,62,False,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,60-69,0,0,0,0,1,0


In [162]:
customers_df.drop(columns=['age','age_range'], inplace=True)

In [163]:
customers_df.head()

,Unnamed: 0,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,0,C1,F,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,0,0,0,0,0,1
1,1,C2,M,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,0,0,0,0,0,1
2,2,C3,M,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,0,1,0,0,0,0
3,3,C4,M,False,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,0,0,0,0,1,0
4,4,C5,M,False,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,0,0,0,0,1,0


In [164]:
customers_df['sex'] = customers_df['sex'].map({"F":0,"M":1})

In [165]:
customers_df.head()

,Unnamed: 0,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,0,C1,0,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,0,0,0,0,0,1
1,1,C2,1,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,0,0,0,0,0,1
2,2,C3,1,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,0,1,0,0,0,0
3,3,C4,1,False,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,0,0,0,0,1,0
4,4,C5,1,False,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,0,0,0,0,1,0


In [166]:
customers_df['is_married'] = customers_df['is_married'].astype('int')

In [167]:
customers_df.head()

,Unnamed: 0,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,0,C1,0,1,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,0,0,0,0,0,1
1,1,C2,1,1,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,0,0,0,0,0,1
2,2,C3,1,1,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,0,1,0,0,0,0
3,3,C4,1,0,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,0,0,0,0,1,0
4,4,C5,1,0,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,0,0,0,0,1,0


In [168]:
customers_df['active_since'] = pd.to_datetime(customers_df['active_since'], format="%Y-%m-%d %H:%M:%S")

In [169]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    10000 non-null  int64         
 1   customer_id   10000 non-null  object        
 2   sex           10000 non-null  int64         
 3   is_married    10000 non-null  int64         
 4   active_since  10000 non-null  datetime64[ns]
 5   event_time    10000 non-null  object        
 6   age_18-29     10000 non-null  int64         
 7   age_30-39     10000 non-null  int64         
 8   age_40-49     10000 non-null  int64         
 9   age_50-59     10000 non-null  int64         
 10  age_60-69     10000 non-null  int64         
 11  age_70-plus   10000 non-null  int64         
dtypes: datetime64[ns](1), int64(9), object(2)
memory usage: 937.6+ KB


In [170]:
def get_delta_days(datetime) -> int :
    today = date.today()
    delta = today - datetime.date()
    return delta.days

In [171]:
customers_df['n_days_active'] = customers_df['active_since'].apply(lambda x: get_delta_days(x))

In [172]:
customers_df.head()

,Unnamed: 0,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,0,C1,0,1,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,2345
1,1,C2,1,1,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,3303
2,2,C3,1,1,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,2818
3,3,C4,1,0,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,3188
4,4,C5,1,0,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,3057


In [173]:
customers_df['n_days_active'] = min_max_scaler.fit_transform(customers_df[['n_days_active']])
customers_df.drop("active_since",axis=1, inplace=True)

In [174]:
customers_df = customers_df.drop(columns=['Unnamed: 0'], errors='ignore')

In [175]:
customers_df.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,0.203425
1,C2,1,1,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,0.859589
2,C3,1,1,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,0.527397
3,C4,1,0,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,0.780822
4,C5,1,0,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,0.691096


In [176]:
customers_df.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,0.203425
1,C2,1,1,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,0.859589
2,C3,1,1,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,0.527397
3,C4,1,0,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,0.780822
4,C5,1,0,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,0.691096


In [177]:
customers_df.to_csv("transformed/customers.csv", index=False)

## Step 2: Transform Orders Data

In [178]:
orders_df = pd.read_csv("raw/orders.csv")

In [179]:
orders_df.head()

,Unnamed: 0,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time
0,0,O1,C9765,P11660,58.84,0,2021-01-10 10:31:27,2024-05-02T05:39:17.172Z
1,1,O2,C3674,P6868,71.08,0,2020-03-20 16:47:36,2024-05-02T05:39:17.172Z
2,2,O3,C2139,P4749,57.17,1,2020-11-01 22:09:22,2024-05-02T05:39:17.172Z
3,3,O4,C7794,P542,5.35,1,2020-03-22 10:10:38,2024-05-02T05:39:17.172Z
4,4,O5,C2229,P7605,47.85,1,2021-01-14 01:08:56,2024-05-02T05:39:17.172Z


In [180]:
orders_df['purchased_on'] =  pd.to_datetime(orders_df['purchased_on'], format='%Y-%m-%d %H:%M:%S')
orders_df['n_days_since_last_purchase'] = orders_df['purchased_on'].apply(lambda x: get_delta_days(x))

In [181]:
orders_df.head()

,Unnamed: 0,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time,n_days_since_last_purchase
0,0,O1,C9765,P11660,58.84,0,2021-01-10 10:31:27,2024-05-02T05:39:17.172Z,1672
1,1,O2,C3674,P6868,71.08,0,2020-03-20 16:47:36,2024-05-02T05:39:17.172Z,1968
2,2,O3,C2139,P4749,57.17,1,2020-11-01 22:09:22,2024-05-02T05:39:17.172Z,1742
3,3,O4,C7794,P542,5.35,1,2020-03-22 10:10:38,2024-05-02T05:39:17.172Z,1966
4,4,O5,C2229,P7605,47.85,1,2021-01-14 01:08:56,2024-05-02T05:39:17.172Z,1668


In [182]:
orders_df['n_days_since_last_purchase'] = min_max_scaler.fit_transform(orders_df[['n_days_since_last_purchase']])
orders_df.head()

,Unnamed: 0,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time,n_days_since_last_purchase
0,0,O1,C9765,P11660,58.84,0,2021-01-10 10:31:27,2024-05-02T05:39:17.172Z,0.273256
1,1,O2,C3674,P6868,71.08,0,2020-03-20 16:47:36,2024-05-02T05:39:17.172Z,0.846899
2,2,O3,C2139,P4749,57.17,1,2020-11-01 22:09:22,2024-05-02T05:39:17.172Z,0.408915
3,3,O4,C7794,P542,5.35,1,2020-03-22 10:10:38,2024-05-02T05:39:17.172Z,0.843023
4,4,O5,C2229,P7605,47.85,1,2021-01-14 01:08:56,2024-05-02T05:39:17.172Z,0.265504


In [183]:
orders_df['purchase_amount'] = min_max_scaler.fit_transform(orders_df[['purchase_amount']])

In [184]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Unnamed: 0                  100000 non-null  int64         
 1   order_id                    100000 non-null  object        
 2   customer_id                 100000 non-null  object        
 3   product_id                  100000 non-null  object        
 4   purchase_amount             100000 non-null  float64       
 5   is_reordered                100000 non-null  int64         
 6   purchased_on                100000 non-null  datetime64[ns]
 7   event_time                  100000 non-null  object        
 8   n_days_since_last_purchase  100000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 6.9+ MB


In [185]:
orders_df['is_reordered'].value_counts()

is_reordered
1    66509
0    33491
Name: count, dtype: int64

In [186]:
orders_df.head()

,Unnamed: 0,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time,n_days_since_last_purchase
0,0,O1,C9765,P11660,0.572673,0,2021-01-10 10:31:27,2024-05-02T05:39:17.172Z,0.273256
1,1,O2,C3674,P6868,0.693861,0,2020-03-20 16:47:36,2024-05-02T05:39:17.172Z,0.846899
2,2,O3,C2139,P4749,0.556139,1,2020-11-01 22:09:22,2024-05-02T05:39:17.172Z,0.408915
3,3,O4,C7794,P542,0.043069,1,2020-03-22 10:10:38,2024-05-02T05:39:17.172Z,0.843023
4,4,O5,C2229,P7605,0.463861,1,2021-01-14 01:08:56,2024-05-02T05:39:17.172Z,0.265504


In [187]:
orders_df.drop(columns="purchased_on", inplace=True)
orders_df = orders_df.drop(columns=['Unnamed: 0'], errors='ignore')

In [188]:
orders_df.head()

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase
0,O1,C9765,P11660,0.572673,0,2024-05-02T05:39:17.172Z,0.273256
1,O2,C3674,P6868,0.693861,0,2024-05-02T05:39:17.172Z,0.846899
2,O3,C2139,P4749,0.556139,1,2024-05-02T05:39:17.172Z,0.408915
3,O4,C7794,P542,0.043069,1,2024-05-02T05:39:17.172Z,0.843023
4,O5,C2229,P7605,0.463861,1,2024-05-02T05:39:17.172Z,0.265504


In [189]:
orders_df.to_csv("transformed/orders.csv", index=False)

## Step 3: Feature Store Setup and Configuration

### Step 3.1 Prepare Environment

In [190]:
import sagemaker
import boto3
import sys
import pandas as pd
import numpy as np 
import io 
from sagemaker.session import Session
from sagemaker import get_execution_role

In [191]:
prefix = "customer-feature-store"
role = get_execution_role()
sagemaker_session = Session() 
region = sagemaker_session.boto_region_name
s3_bucket_name = sagemaker_session.default_bucket()

### Step 3.2: Load Transformed Data

In [192]:
# Load the Data 
customer_data = pd.read_csv("transformed/customers.csv") 
orders_data = pd.read_csv("transformed/orders.csv")

In [193]:
customer_data.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,0.203425
1,C2,1,1,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,0.859589
2,C3,1,1,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,0.527397
3,C4,1,0,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,0.780822
4,C5,1,0,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,0.691096


In [194]:
orders_data.head()

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase
0,O1,C9765,P11660,0.572673,0,2024-05-02T05:39:17.172Z,0.273256
1,O2,C3674,P6868,0.693861,0,2024-05-02T05:39:17.172Z,0.846899
2,O3,C2139,P4749,0.556139,1,2024-05-02T05:39:17.172Z,0.408915
3,O4,C7794,P542,0.043069,1,2024-05-02T05:39:17.172Z,0.843023
4,O5,C2229,P7605,0.463861,1,2024-05-02T05:39:17.172Z,0.265504


### Step 3.3: Create Feature Groups

In [203]:
# Create feature group
from time import gmtime, strftime, sleep

customers_feature_group_name = "customers-feature-group" 
orders_feature_group_name = "orders-feature-group"
print(f"Customer Feature Group Name: {customers_feature_group_name}")
print(f"Orders Feature Group Name: {orders_feature_group_name}")

Customer Feature Group Name: customers-feature-group
Orders Feature Group Name: orders-feature-group


In [204]:
from sagemaker.feature_store.feature_group import FeatureGroup

customers_feature_group = FeatureGroup(name=customers_feature_group_name , 
                                       sagemaker_session = sagemaker_session)

orders_feature_group = FeatureGroup(name=orders_feature_group_name, description="Test", sagemaker_session = sagemaker_session)

In [205]:
record_identifier_feature_name = "customer_id"

In [206]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customer_id    10000 non-null  object 
 1   sex            10000 non-null  int64  
 2   is_married     10000 non-null  int64  
 3   event_time     10000 non-null  object 
 4   age_18-29      10000 non-null  int64  
 5   age_30-39      10000 non-null  int64  
 6   age_40-49      10000 non-null  int64  
 7   age_50-59      10000 non-null  int64  
 8   age_60-69      10000 non-null  int64  
 9   age_70-plus    10000 non-null  int64  
 10  n_days_active  10000 non-null  float64
dtypes: float64(1), int64(8), object(2)
memory usage: 859.5+ KB


In [207]:
orders_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_id                    100000 non-null  object 
 1   customer_id                 100000 non-null  object 
 2   product_id                  100000 non-null  object 
 3   purchase_amount             100000 non-null  float64
 4   is_reordered                100000 non-null  int64  
 5   event_time                  100000 non-null  object 
 6   n_days_since_last_purchase  100000 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 5.3+ MB


In [208]:
# Load Customer Feature definitions
customers_feature_group.load_feature_definitions(data_frame = customer_data)

[FeatureDefinition(feature_name='customer_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='sex', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='is_married', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='age_18-29', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_30-39', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_40-49', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_50-59', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_60-69

In [209]:
# Load Order Feature definitions
orders_feature_group.load_feature_definitions(data_frame = orders_data)

[FeatureDefinition(feature_name='order_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='customer_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='product_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='purchase_amount', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='is_reordered', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='n_days_since_last_purchase', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None)]

In [210]:
customers_feature_group.create(s3_uri = f"s3://{s3_bucket_name}/{prefix}",
                               record_identifier_name=record_identifier_feature_name, event_time_feature_name="event_time",
                               role_arn=role,
                               enable_online_store=True)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:590183953604:feature-group/customers-feature-group',
 'ResponseMetadata': {'RequestId': 'dc0a4e7e-aeb5-4383-9478-63cf0da314e5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dc0a4e7e-aeb5-4383-9478-63cf0da314e5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Sat, 09 Aug 2025 20:01:55 GMT'},
  'RetryAttempts': 0}}

In [211]:
orders_feature_group.create(s3_uri = f"s3://{s3_bucket_name}/{prefix}",
                               record_identifier_name=record_identifier_feature_name, event_time_feature_name="event_time",
                                role_arn=role,
                                enable_online_store=True)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:590183953604:feature-group/orders-feature-group',
 'ResponseMetadata': {'RequestId': '9d0be86f-f332-4012-aa7f-6f0e3e7eca0a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9d0be86f-f332-4012-aa7f-6f0e3e7eca0a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Sat, 09 Aug 2025 20:01:58 GMT'},
  'RetryAttempts': 2}}

In [212]:
customers_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:590183953604:feature-group/customers-feature-group',
 'FeatureGroupName': 'customers-feature-group',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'String'},
  {'FeatureName': 'sex', 'FeatureType': 'Integral'},
  {'FeatureName': 'is_married', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'},
  {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2025, 8, 9, 20, 1, 55, tzinfo=tzlocal()),
 'OnlineSt

In [213]:
orders_feature_group.describe() 

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:590183953604:feature-group/orders-feature-group',
 'FeatureGroupName': 'orders-feature-group',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'order_id', 'FeatureType': 'String'},
  {'FeatureName': 'customer_id', 'FeatureType': 'String'},
  {'FeatureName': 'product_id', 'FeatureType': 'String'},
  {'FeatureName': 'purchase_amount', 'FeatureType': 'Fractional'},
  {'FeatureName': 'is_reordered', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'n_days_since_last_purchase', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2025, 8, 9, 20, 1, 57, 666000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-us-east-1-590183953604/customer-feature-store',
   'ResolvedOutputS3Uri': 's3://sagemaker-us-east-1-5901

### Step 3.4: Add Metadata to Features

In [215]:
# description and parameters

from sagemaker.feature_store.inputs import FeatureParameter

customers_feature_group.update_feature_metadata(feature_name = "customer_id", description= "The ID of the customer, it is also part of Order feature group",
                                                parameter_additions=[FeatureParameter("idType","primarykey")])

{'ResponseMetadata': {'RequestId': 'e63bf0bf-9e37-4579-b2e0-2205a26e792b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e63bf0bf-9e37-4579-b2e0-2205a26e792b',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 09 Aug 2025 20:03:59 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [216]:
customers_feature_group.describe_feature_metadata("customer_id")

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:590183953604:feature-group/customers-feature-group',
 'FeatureGroupName': 'customers-feature-group',
 'FeatureName': 'customer_id',
 'FeatureType': 'String',
 'CreationTime': datetime.datetime(2025, 8, 9, 20, 1, 55, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 8, 9, 20, 3, 59, 743000, tzinfo=tzlocal()),
 'Description': 'The ID of the customer, it is also part of Order feature group',
 'Parameters': [{'Key': 'idType', 'Value': 'primarykey'}],
 'ResponseMetadata': {'RequestId': '089f7892-a3cc-47a3-89da-e8628c01bace',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '089f7892-a3cc-47a3-89da-e8628c01bace',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '393',
   'date': 'Sat, 09 Aug 2025 20:03:59 GMT'},
  'RetryAttempts': 0}}

In [217]:
sagemaker_session.boto_session.client("sagemaker", region_name=region).search(
    Resource="FeatureMetadata",
    SearchExpression={
        "Filters":[
            {
                "Name": "FeatureGroupName",
                "Operator": "Contains",
                "Value": "customers-fg"
            },
            {"Name": "Parameters.idType", "Operator": "Equals", "Value": "primarykey"}
        ]
    })

{'Results': [{'FeatureMetadata': {'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:590183953604:feature-group/customers-fg-09-19-26-55',
    'FeatureGroupName': 'customers-fg-09-19-26-55',
    'FeatureName': 'customer_id',
    'FeatureType': 'String',
    'CreationTime': datetime.datetime(2025, 8, 9, 19, 26, 55, tzinfo=tzlocal()),
    'LastModifiedTime': datetime.datetime(2025, 8, 9, 19, 58, 2, tzinfo=tzlocal()),
    'Description': 'The ID of the customer, it is also part of Order feature group',
    'Parameters': [{'Key': 'idType', 'Value': 'primarykey'}]}}],
 'ResponseMetadata': {'RequestId': '84d6739c-8933-4023-a8f5-1b3b03c340fc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '84d6739c-8933-4023-a8f5-1b3b03c340fc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '507',
   'date': 'Sat, 09 Aug 2025 20:03:59 GMT'},
  'RetryAttempts': 0}}

### Step 3.5: Ingest data into Feature Store

In [218]:
customers_feature_group.ingest(data_frame = customer_data, max_workers=3,wait=True)

IngestionManagerPandas(feature_group_name='customers-feature-group', feature_definitions={'customer_id': {'FeatureName': 'customer_id', 'FeatureType': 'String'}, 'sex': {'FeatureName': 'sex', 'FeatureType': 'Integral'}, 'is_married': {'FeatureName': 'is_married', 'FeatureType': 'Integral'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'age_18-29': {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'}, 'age_30-39': {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'}, 'age_40-49': {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'}, 'age_50-59': {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'}, 'age_60-69': {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'}, 'age_70-plus': {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'}, 'n_days_active': {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'}}, sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f833788dd90>, sagemaker_session=<sagemaker.session.Session ob

In [ ]:
orders_feature_group.ingest(data_frame = orders_data, max_workers=3,wait=True)

### Step 3.6: Validate Feature Store Data

In [ ]:
customer_id = "C1"
sample_record = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).get_record(
    FeatureGroupName=customers_feature_group_name, RecordIdentifierValueAsString=str(customer_id)
)

In [ ]:
sample_record

{'ResponseMetadata': {'RequestId': '4061fcf9-0612-42cd-a75f-aea0481208ba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4061fcf9-0612-42cd-a75f-aea0481208ba',
   'content-type': 'application/json',
   'content-length': '877',
   'date': 'Sat, 09 Aug 2025 20:12:27 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'customer_id', 'ValueAsString': 'C1'},
  {'FeatureName': 'sex', 'ValueAsString': '0'},
  {'FeatureName': 'is_married', 'ValueAsString': '1'},
  {'FeatureName': 'event_time', 'ValueAsString': '2024-05-02T05:39:10.965Z'},
  {'FeatureName': 'age_18-29', 'ValueAsString': '0'},
  {'FeatureName': 'age_30-39', 'ValueAsString': '0'},
  {'FeatureName': 'age_40-49', 'ValueAsString': '0'},
  {'FeatureName': 'age_50-59', 'ValueAsString': '0'},
  {'FeatureName': 'age_60-69', 'ValueAsString': '0'},
  {'FeatureName': 'age_70-plus', 'ValueAsString': '1'},
  {'FeatureName': 'n_days_active', 'ValueAsString': '0.2034246575342466'}]}

In [ ]:
all_records = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": customers_feature_group_name,
            "RecordIdentifiersValueAsString": ["C1","C2"],
        },
        {
            "FeatureGroupName": orders_feature_group_name,
            "RecordIdentifiersValueAsString": ["C1","C2"],
        },
    ]
)

In [ ]:
all_records

{'ResponseMetadata': {'RequestId': 'ad0ec71b-d5eb-4f0b-a2bf-1d2448298799',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ad0ec71b-d5eb-4f0b-a2bf-1d2448298799',
   'content-type': 'application/json',
   'content-length': '3404',
   'date': 'Sat, 09 Aug 2025 20:12:27 GMT'},
  'RetryAttempts': 0},
 'Records': [{'FeatureGroupName': 'orders-feature-group',
   'RecordIdentifierValueAsString': 'C1',
   'Record': [{'FeatureName': 'order_id', 'ValueAsString': 'O99056'},
    {'FeatureName': 'customer_id', 'ValueAsString': 'C1'},
    {'FeatureName': 'product_id', 'ValueAsString': 'P5567'},
    {'FeatureName': 'purchase_amount', 'ValueAsString': '0.6516831683168316'},
    {'FeatureName': 'is_reordered', 'ValueAsString': '1'},
    {'FeatureName': 'event_time', 'ValueAsString': '2024-05-02T05:39:18.632Z'},
    {'FeatureName': 'n_days_since_last_purchase',
     'ValueAsString': '0.5484496124031009'}]},
  {'FeatureGroupName': 'orders-feature-group',
   'RecordIdentifierValueAsString'